# Multiclass Text Classification with TF-IDF and AFTER SMOTE

In [1]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [2]:
import pandas as pd #Data manipulation
import numpy as np
from scipy.stats import randint
import seaborn as sns # used for plot interactive graph.
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
#import warnings
#warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
df=pd.read_csv('/content/drive/MyDrive/movies.csv')

### Exploration and Preprocessing

In [4]:
df.head(10) #this way you can display columns as rows.

Unnamed: 0        genre                                        description
0           0        drama  Listening in to a conversation between his doc...
1           1     thriller  A brother and sister with a past incestuous re...
2           2        adult  As the bus empties the students for their fiel...
3           3        drama  To help their unemployed father make ends meet...
4           4        drama  The film's title refers not only to the un-rec...
5           5  documentary  Quality Control consists of a series of 16mm s...
6           6       comedy  In tough economic times Max and Joey have all ...
7           7        crime  Ron Petrie (Keanu Reeves) is a troubled teen w...
8           8   reality-tv  A sudden calamitous event, causing great loss ...
9           9       horror  Four high school students embark on a terrifyi...

In [5]:
# Create a new dataframe with two columns
df1 = df[['genre','description']].copy()

# Remove missing values (NaN)
df1 = df1[pd.notnull(df1['description'])]

# Renaming second column for a simpler name
#df1.columns = ['Product', 'Complaint']

df1.shape

(54214, 2)

In [6]:
# Percentage of complaints that had text field
total = df1['description'].notnull().sum()
round((total/len(df)*100),1)

100.0

In [7]:
pd.DataFrame(df.genre.unique()).values

array([['drama'],
       ['thriller'],
       ['adult'],
       ['documentary'],
       ['comedy'],
       ['crime'],
       ['reality-tv'],
       ['horror'],
       ['sport'],
       ['animation'],
       ['action'],
       ['fantasy'],
       ['short'],
       ['sci-fi'],
       ['music'],
       ['adventure'],
       ['talk-show'],
       ['western'],
       ['family'],
       ['mystery'],
       ['history'],
       ['news'],
       ['biography'],
       ['romance'],
       ['game-show'],
       ['musical'],
       ['war']], dtype=object)

In [8]:
# let's work with a smaller smaple of the data to speed things up
df2 = df1.sample(40000, random_state=2).copy()

In [9]:
pd.DataFrame(df2.genre.unique())
df2.genre.value_counts()

drama          10032
documentary     9701
comedy          5456
short           3757
horror          1619
thriller        1160
action           988
western          770
reality-tv       660
family           571
adventure        565
music            526
romance          481
sci-fi           456
adult            435
crime            379
animation        377
sport            330
talk-show        282
fantasy          250
mystery          248
musical          199
biography        197
history          195
game-show        143
news             131
war               92
Name: genre, dtype: int64

In [10]:
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from collections import Counter

# Assuming df2 is your DataFrame

# Replace less frequent genres with 'other'
genre_counts = df2.genre.value_counts()
genres_to_replace = genre_counts[genre_counts < 200].index
df2.loc[df2.genre.isin(genres_to_replace), 'genre'] = 'other'

# Check the updated genre counts
print(df2.genre.value_counts())

# Proceed with feature extraction
tfidf = TfidfVectorizer(max_features=1000)  # Adjust parameters as needed
X = tfidf.fit_transform(df2['description'])
y = df2['genre']

# Apply SMOTE
smote = SMOTE()
X_res, y_res = smote.fit_resample(X, y)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42)

# Train the LinearSVC model
model = LinearSVC()
model.fit(X_train, y_train)

# Predict and evaluate the model
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))


drama          10032
documentary     9701
comedy          5456
short           3757
horror          1619
thriller        1160
action           988
other            957
western          770
reality-tv       660
family           571
adventure        565
music            526
romance          481
sci-fi           456
adult            435
crime            379
animation        377
sport            330
talk-show        282
fantasy          250
mystery          248
Name: genre, dtype: int64
              precision    recall  f1-score   support

      action       0.73      0.69      0.71      2020
       adult       0.87      0.98      0.92      1924
   adventure       0.74      0.77      0.76      1976
   animation       0.81      0.93      0.87      1961
      comedy       0.61      0.40      0.48      2073
       crime       0.81      0.93      0.87      1990
 documentary       0.59      0.51      0.55      1970
       drama       0.44      0.18      0.25      1971
      family       0.76  

In [11]:

# Calculate and print the accuracy
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

# Print genre counts after applying SMOTE
print("Genre counts after SMOTE:")
print(Counter(y_res))

Accuracy: 0.7846673161006774
Genre counts after SMOTE:
Counter({'drama': 10032, 'short': 10032, 'comedy': 10032, 'horror': 10032, 'documentary': 10032, 'reality-tv': 10032, 'thriller': 10032, 'romance': 10032, 'sci-fi': 10032, 'animation': 10032, 'adult': 10032, 'other': 10032, 'family': 10032, 'fantasy': 10032, 'sport': 10032, 'action': 10032, 'adventure': 10032, 'music': 10032, 'western': 10032, 'crime': 10032, 'talk-show': 10032, 'mystery': 10032})
